In [ ]:
!pip install PyPDF2

In [ ]:
import PyPDF2

# Open the PDF file
with open('/content/RBI.PDF', 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)

    # Iterate through each page and extract text
    pdf_text = "" # Initialize an empty string to store the extracted text
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        page_text = page.extract_text()
        pdf_text += page_text # Append the text of each page to pdf_text

        # Print each page's text with a separator
        print(f"Page {page_num + 1}:\n")
        print(page_text)
        print("\n" + "="*40 + "\n")  # Adds a separator between pages

# Optionally, save all the extracted text to a file
with open('output.txt', 'w', encoding='utf-8') as text_file:
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text_file.write(f"Page {page_num + 1}:\n\n")
        text_file.write(page.extract_text())
        text_file.write("\n" + "="*40 + "\n")



In [ ]:
!pip install langchain


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define a function to chunk non-table text using Langchain
def chunk_non_table_text_langchain(text, chunk_size=1000, chunk_overlap=200):
    # Create a RecursiveCharacterTextSplitter instance
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )

    # Split the text into lines
    lines = text.split('\n')
    non_table_text = []

    # Collect non-table lines
    for line in lines:
        # Skip table-like lines
        if "|" in line or line.strip().startswith(":"):
            continue
        non_table_text.append(line)

    # Join non-table lines back into a single string
    non_table_text_str = '\n'.join(non_table_text)

    # Use Langchain's text splitter to create chunks
    chunks = text_splitter.split_text(non_table_text_str)

    return chunks



# Chunk the non-table text
chunks = chunk_non_table_text_langchain(pdf_text)

# Display the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:\n{chunk}\n")


In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load a pre-trained model
model = SentenceTransformer('all-MPNet-base-v2')  # You can choose other models available



# Create embeddings for each chunk
chunk_embeddings = model.encode(chunks, convert_to_numpy=True)



In [ ]:
chunk_embeddings

In [ ]:
!pip install chromadb sentence-transformers


In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

# Initialize ChromaDB client
client = chromadb.Client()

# Create a new collection to store embeddings
collection = client.create_collection("pdf_chunks_embeddings")


In [ ]:
# Load the SentenceTransformer model
model = SentenceTransformer('all-MPNet-base-v2')

# Assume chunks contain the text data split into parts from your PDF
chunks = chunk_non_table_text_langchain(pdf_text)

# Create embeddings for each chunk
chunk_embeddings = model.encode(chunks, convert_to_numpy=True)

# Store embeddings in ChromaDB
for idx, embedding in enumerate(chunk_embeddings):
    collection.add(
        ids=[str(idx)],  # Use a unique identifier for each chunk, here using index
        embeddings=[embedding.tolist()],  # Store the embedding as a list
        metadatas=[{"text": chunks[idx]}]  # Store the actual chunk text as metadata
    )

In [ ]:
# Define a query text
query = "what is point 13 IN Agriculture and Climate Change"

# Generate embedding for the query
query_embedding = model.encode([query], convert_to_numpy=True)

# Search for the most similar chunks in ChromaDB
results = collection.query(
    query_embeddings=[query_embedding[0].tolist()], # Convert the NumPy array to a list
    n_results=3  # Retrieve top 3 most similar chunks
)

# Display the results
for result in results["metadatas"][0]: # Access the first list within "metadatas"
    print("Similar chunk:", result["text"])
    print()


In [ ]:
!pip install torch

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()


In [ ]:
!pip install flash_attn

In [ ]:
# Load model directly

from transformers import AutoTokenizer, AutoModelForCausalLM

#tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B", trust_remote_code=True).half() # Remove .cuda() to avoid using GPU
# Check if CUDA is available and move model to GPU if it is
if torch.cuda.is_available():
    model.to('cuda')

In [ ]:
def generate_prompt(chunk, query):
    # Create the prompt template
    prompt = f"Context: {chunk}\n\nQuery: {query}\n\nResponse:"
    return prompt

def get_llm_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=32000, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Create prompt for LLM
prompt = generate_prompt(chunk, query) # Use chunk instead of most_relevant_chunk
response = get_llm_response(prompt)

print("LLM Response:")
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
